In [6]:
from PSO import *
from testfunction import *

In [8]:
opt = PSO(size = 25, 
          n_iter=100, 
          position_boundary=[(-1, 1, 'float'), (-1, 1, 'float')], 
          velocity_boundary=[(0, 0.01, 'float'), (0, 0.01, 'float')], 
          objective_function=sphere, 
          objective_type='minimum', 
          C1=0.5, 
          C2=0.75, 
          random_state=42, 
          save_history=True)

opt.simulate()

iteration:  0  | best particle:  [0.02788669 0.00870428]  | best fitness:  0.0008534317778613043
iteration:  1  | best particle:  [-0.02271711  0.00746379]  | best fitness:  0.0005717752423049662
iteration:  2  | best particle:  [0.30706545 0.01222017]  | best fitness:  0.09443852260114853
iteration:  3  | best particle:  [-0.05953985  0.00802579]  | best fitness:  0.0036094066512728095
iteration:  4  | best particle:  [-0.01844129  0.0102458 ]  | best fitness:  0.0004450575305675424
iteration:  5  | best particle:  [ 0.03370576 -0.00375607]  | best fitness:  0.0011501863689752565
iteration:  6  | best particle:  [-0.06946062  0.00338405]  | best fitness:  0.00483622929789233
iteration:  7  | best particle:  [-0.00588162 -0.01806756]  | best fitness:  0.000361030123743218
iteration:  8  | best particle:  [ 0.01313006 -0.00369431]  | best fitness:  0.00018604644260269385
iteration:  9  | best particle:  [-0.06641818 -0.01180957]  | best fitness:  0.004550840792039352
iteration:  10  | b

In [3]:
def unpack(list_of_history):
    unpacked = []
    for lst in list_of_history:
        coordinates = ([x.position for x in lst])
        unpacked.append(coordinates)
    return unpacked

def get_fit(list_of_coordinates, objective_function):
    fitness_history = []
    for lst in list_of_coordinates:
        fitness = [objective_function(x) for x in lst]
        fitness_history.append(fitness)
    return fitness_history

In [4]:
coord = unpack(opt.history)
fitn = get_fit(coord, sphere)

In [5]:
unpack(opt.history)

),
  array([  -2327.23667453, -105196.84456791])],
 [array([506.12798608,  16.84434375]),
  array([1737.29772026, -596.95846451]),
  array([-738.22017235, -364.80365769]),
  array([-1127.32058967,  -504.00908637]),
  array([402.62521648, -98.35550225]),
  array([290.49335296, 454.47102999]),
  array([403.35527978, 227.55091193]),
  array([-227.13788712, 2556.61272773]),
  array([ 1136.30778857, -7422.61278032]),
  array([-4471.21822853,  -263.89750949]),
  array([ 2194.58692653, -1022.29989368]),
  array([286.14658266, 463.69402424]),
  array([ 4777.18076118, -1444.13555381]),
  array([2497.44952014, 1007.72601378]),
  array([ 1375.28014505, -2817.08471039]),
  array([-899.75644833,  356.01925288]),
  array([  84.65705038, -478.94336354]),
  array([ 1745.52157008, -5458.52211546]),
  array([4621.66218336,  360.0539792 ]),
  array([ -553.19500173, -1678.9311899 ]),
  array([-12579.80297   ,   -833.90515947]),
  array([-4292.13902222,   277.27125334]),
  array([-1735.98333112,  -717.9585

In [309]:
class Particle:
    def __init__(self, initial_position, initial_velocity, position_boundary, velocity_boundary):
        self.position = np.array(initial_position)
        self.velocity = np.array(initial_velocity)
        self.position_boundary = position_boundary
        self.velocity_boundary = velocity_boundary
        self.best_position = None
        self.initialized = False
        self.dim = len(initial_position)
    
    def fit(self, fitness_function):
        self.fitness = fitness_function(self.position)
        if (self.initialized == False) or (self.fitness < fitness_function(self.best_position)):
            self.best_position = self.position.copy()
            self.best_fitness = self.fitness.copy()
            self.initialized = True

    def update_attributes(self, w_inertia, C1, C2, global_best_position):
        w_inertia = 0.5
        C1 = 1
        C2 = 2
        rand1 = np.random.rand(len(self.position))
        rand2 = np.random.rand(len(self.position))
        cognitive_velocity = C1*rand1*(self.best_position - self.position)
        social_velocity = C2*rand2*(global_best_position - self.position)
        self.velocity = w_inertia*self.velocity + cognitive_velocity + social_velocity
        self.position = self.position + self.velocity
        for i in range(self.dim):
            if self.position[i] > self.position_boundary[i][1]:
                self.position[i] = self.position_boundary[i][1]
            if self.position[i] < self.position_boundary[i][0]:
                self.position[i] = self.position_boundary[i][0]
    
class PSO:
    def __init__(self, size, n_iter, fitness_function, position_boundary, velocity_boundary, w_inertia, C1, C2, random_state, save_history):
        self.size = size
        self.n_iter = n_iter
        self.fitness_function = fitness_function
        self.position_boundary = position_boundary
        self.velocity_boundary = velocity_boundary
        self.w_inertia = w_inertia
        self.C1 = C1
        self.C2 = C2
        self.save_history = save_history
        np.random.seed(random_state)

    def simulate(self):
        # Initialize
        swarm = []
        for i in range(self.size):
            position = []
            velocity = []
            for pos_bound in self.position_boundary:
                pos_ = np.random.uniform(pos_bound[0], pos_bound[1])
                position.append(pos_)
            for vel_bound in self.velocity_boundary: 
                vel_ = np.random.uniform(vel_bound[0], vel_bound[1])
                velocity.append(vel_)
            initial_particle = Particle(position, velocity, self.position_boundary, self.velocity_boundary)
            swarm.append(initial_particle)

        if self.save_history == True:
            swarm_history = []
            swarm_history.append([x.position for x in swarm])

        # Optimization
        iteration = 0
        global_best_fitness = -1
        while iteration < self.n_iter:
            temp = []

            for j in range(self.size):
                swarm[j].fit(self.fitness_function)
                if (swarm[j].fitness < global_best_fitness) or global_best_fitness == -1:
                    global_best_position = swarm[j].position
                    global_best_fitness = swarm[j].fitness

            for j in range(self.size):
                swarm[j].update_attributes(self.w_inertia, self.C1, self.C2, global_best_position)
                temp.append(swarm[j].position)

            iteration += 1
            print(f'iteration: {iteration} | best_position : {global_best_position} | best_fit : {global_best_fitness}')
            if self.save_history == True:
                swarm_history.append(temp)

        self.sol = global_best_position
        if self.save_history == True:
            self.swarm_history = swarm_history

In [310]:
opt = PSO(size=10, 
          n_iter=10, 
          fitness_function=sphere, 
          position_boundary=[(0.0, 1.0), (0.0, 1.0), (0.0, 1.0)], 
          velocity_boundary=[(-0.5, 0.5), (-0.5, 0.5), (-0.5, 0.5)],
          w_inertia = 0.25,
          C1 = 0.75,
          C2 = 1,
          random_state = 42,
          save_history = True)

In [311]:
opt.simulate()

iteration: 1 | best_position : [0.60754485 0.17052412 0.06505159] | best_fit : 0.40242093358118186
iteration: 2 | best_position : [0.34005735 0.         0.        ] | best_fit : 0.11563900104111205
iteration: 3 | best_position : [0.09914277 0.         0.        ] | best_fit : 0.009829289181043662
iteration: 4 | best_position : [0. 0. 0.] | best_fit : 0.0
iteration: 5 | best_position : [0. 0. 0.] | best_fit : 0.0
iteration: 6 | best_position : [0. 0. 0.] | best_fit : 0.0
iteration: 7 | best_position : [0. 0. 0.] | best_fit : 0.0
iteration: 8 | best_position : [0. 0. 0.] | best_fit : 0.0
iteration: 9 | best_position : [0. 0. 0.] | best_fit : 0.0
iteration: 10 | best_position : [0. 0. 0.] | best_fit : 0.0


In [312]:
np.save('swarm_history.npy', opt.swarm_history)

[[array([0.37454012, 0.95071431, 0.73199394]),
  array([0.05808361, 0.86617615, 0.60111501]),
  array([0.83244264, 0.21233911, 0.18182497]),
  array([0.43194502, 0.29122914, 0.61185289]),
  array([0.45606998, 0.78517596, 0.19967378]),
  array([0.60754485, 0.17052412, 0.06505159]),
  array([0.30461377, 0.09767211, 0.68423303]),
  array([0.03438852, 0.9093204 , 0.25877998]),
  array([0.54671028, 0.18485446, 0.96958463]),
  array([0.59789998, 0.92187424, 0.0884925 ])],
 [array([0.59011979, 0.34035893, 0.        ]),
  array([1.        , 0.        , 0.62302152]),
  array([0.3462407 , 0.04995891, 0.1769103 ]),
  array([0.47059404, 0.10741936, 0.47552624]),
  array([0.65633507, 0.        , 0.        ]),
  array([0.83198762, 0.40334014, 0.21925027]),
  array([0.34005735, 0.        , 0.        ]),
  array([0.4014165 , 0.20879719, 0.        ]),
  array([0.70389235, 0.37795817, 0.        ]),
  array([0.4494903, 0.       , 0.       ])],
 [array([0.64286992, 0.        , 0.        ]),
  array([0.149